In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns

# 0. Read data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Engenharia de Trafego/GG_results_full2.csv')

In [ ]:
mapping = {
    'motorcycle': 'VP',
    'car': 'VP',
    'bus': 'CO',
    'truck': 'CO',
    'train': 'SR/RE'
}
df['type_renamed'] = df['type'].map(mapping)
df.head()

,id,type,initial_frame,detection_duration,lenght,type_renamed
0,0,car,1,18,85.962891,VP
1,1,truck,1,42,325.190979,CO
2,2,truck,1,2,328.913086,CO
3,3,car,3,6,96.729248,VP
4,4,car,35,19,167.295029,VP


# 2.b. Apresente as contagens por classe de veículo e intervalo de 15 minutos em uma tabela e em um gráfico de colunas;

In [ ]:
def minutes_to_frames(minutes):
    fps = 30
    return min(minutes * 60 * fps, 107933) # 107920 is the number of frames in the video

intervals = [
    [0, minutes_to_frames(15), '10:00 - 10:15'],
    [minutes_to_frames(15), minutes_to_frames(30), '10:15 - 10:30'],
    [minutes_to_frames(30), minutes_to_frames(45), '10:30 - 10:45'],
    [minutes_to_frames(45), minutes_to_frames(60), '10:45 - 11:00'],
]

for interval in intervals:
    start, end, label = interval
    temp = df[(df['initial_frame'] >= start) & (df['initial_frame'] <= end)]
    temp.loc[:, 'count'] = 1
    interval.append(temp)

import plotly.express as px

def plot_vehicles_interval(interval):
    print('Time interval: ', interval[2])
    df = interval[3][['type_renamed', 'count']].groupby(['type_renamed']).count().reset_index()
    fig = px.bar(df, 
                 x='type_renamed', 
                 y='count', 
                 title="Contagem de Veículos no Intervalo " + str(interval[2]), 
                 labels={
                     'type_renamed': "Tipo de Veículo",
                     'count': 'Contagem no Intervalo'
                 },
                 text_auto=True)
    fig.update_traces(marker_color='#ef233c')
    # fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    # fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
    fig.show()
    return df

<ipython-input-21-c2014537b77f>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-c2014537b77f>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-c2014537b77f>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-c201

In [ ]:
for i in range(4):
    r = plot_vehicles_interval(intervals[i])
    r


Time interval:  10:00 - 10:15


Time interval:  10:15 - 10:30


Time interval:  10:30 - 10:45


Time interval:  10:45 - 11:00


# 2.c Converta os volumes em veículos de passeio equivalentes pelos fatores de conversão do DNIT indicados a seguir e apresente os resultados em intervalos de 15 minutos em uma tabela e em um gráfico de colunas;

In [ ]:
convertion_factor = {
    'VP': 1,
    'CO': 1.5,
    'SR/RE': 2.
}


In [ ]:
convertion_factor = {
    'VP': 1,
    'CO': 1.5,
    'SR/RE': 2.
}

def plot_vehicles_vp_eq_interval(interval):
    print('Time interval: ', interval[2])
    df = interval[3][['type_renamed', 'count']].groupby(['type_renamed']).count().reset_index()
    df['count'] = df['count'] * df['type_renamed'].map(convertion_factor)
    df['count (VP equivalent)'] = df['count']
    df = df[['type_renamed', 'count (VP equivalent)']]
    fig = px.bar(df, x='type_renamed', y='count (VP equivalent)', 
                 labels= {'type_renamed': 'Tipo de Veículo', 'count (VP equivalent)': 'Contagem em VP Equivalente'},
                 title="Conversão em Veículo de Passeio Equivalente no Intervalo: "+str(interval[2]), text_auto=True)
    fig.update_traces(marker_color='#ef233c')
    fig.show()
    return df

for i in range(4):
    plot_vehicles_vp_eq_interval(intervals[i])

Time interval:  10:00 - 10:15


Time interval:  10:15 - 10:30


Time interval:  10:30 - 10:45


Time interval:  10:45 - 11:00


# 2.d. Determine os fluxos horários nos intervalos de 15 minutos em veículos de passeio equivalentes e apresente os resultados em uma tabela e em um gráfico de colunas;

In [ ]:
def flux(intervals):
    fluxes = []
    for interval in intervals:

        df = interval[3][['type_renamed', 'count']].groupby(['type_renamed']).count().reset_index()
        df['count'] = df['count'] * df['type_renamed'].map(convertion_factor)
        df['count (VP equivalent)'] = df['count']
        df = df[['type_renamed', 'count (VP equivalent)']]
        flux = df['count (VP equivalent)'].sum()*4
        fluxes.append([interval[2], flux])
    fluxes = pd.DataFrame(fluxes, columns=['interval', 'flux (cars/hr)'])
    fig = px.bar(fluxes, x='interval', y= 'flux (cars/hr)',
                 labels={'interval': 'Intervalos',
                         'flux (cars/hr)': 'Fluxo Horário em VP Equivalentes'},
                 title='Fluxos Horários nos Intervalos de 15 Minutos em VP Equivalentes'
                 , text_auto=True)
    fig.update_traces(marker_color='#ef233c')
    fig.show()
    return fluxes

fluxes = flux(intervals)
fluxes

,interval,flux (cars/hr)
0,10:00 - 10:15,6974.0
1,10:15 - 10:30,6824.0
2,10:30 - 10:45,6716.0
3,10:45 - 11:00,6618.0


# 2.e. Calcule o fator hora-pico

In [ ]:

fluxes.columns

Index(['interval', 'flux (cars/hr)'], dtype='object')

In [ ]:
valor_horario = fluxes['flux (cars/hr)'].sum()

fator_hora_pico = valor_horario/fluxes['flux (cars/hr)'].max() * (15/60)

print('Fator hora pico: %.2f'%fator_hora_pico)

Fator hora pico: 0.97


In [ ]:
fluxes['flux (cars/hr)'].max()

6974.0

# 2.f. Calcule a densidade em intervalos de 15 minutos (em veículos de passeio equivalentes/km.faixa) e apresente os resultados em uma tabela e em um gráfico de colunas;

In [ ]:
estimated_road_lenght = 21

density = []
for interval in intervals[:4]:
    final_frame = interval[1]
    s1 = df['initial_frame'] <= final_frame
    s2 = df['initial_frame'] + df['detection_duration'] >= final_frame
    s3 = s1 * s2
    cars = df[s3]
    cars_converted = cars['type_renamed'].map(convertion_factor).sum()
    density.append([interval[2].split()[-1], cars_converted/(estimated_road_lenght/1000)/3])

densities = pd.DataFrame(density, columns=['time', 'density (VP/km.faixa)'])
fig = px.bar(densities, x='time', y='density (VP/km.faixa)', 
             title='Densidade em Intervalos de 15 Minutos [VP/km.faixa]',
             labels={
                 "time": "Intervalos",
                 'density (VP/km.faixa)': 'Densidade de Veículos'
             }
             , text_auto=True)
fig.update_traces(marker_color='#ef233c')
fig.show()

# 2.g. Determine o fluxo horário médio e a densidade média no intervalo de uma hora;

In [ ]:
print('Densidade média %.2f Veículos/km.Faixa'%densities.mean())
print('Fluxo horario médio %.0f Veículos/hora'%valor_horario)

Densidade média 15.87 Veículos/km.Faixa
Fluxo horario médio 27132 Veículos/hora


<ipython-input-44-5e4bfb1a9909>:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

